# 2D Image Segmentation

In this tutorial we will be training a 2D UNet to perform automatic segementation of the blood-pool from Cardiac MRI images

# Set up TensorFlow MRI
If you have not yet installed Pydicom or TensorFlow MRI in your environment you may need to do so now
using `pip`

In [ ]:
!pip install tensorflow-mri pydicom

### Using a GPU
TensorFlow MRI supports CPU and GPU computation. If there is a GPU available in your environment and it is visible to TensorFlow, it will be used automatically.

:::{tip}
In Google Colab, you can enable GPU computation by clicking on
**Runtime > Change runtime type** and selecting **GPU** under
**Hardware accelerator**.
:::

:::{tip}
You can control whether CPU or GPU is used for a particular operation via
the [`tf.device`](https://www.tensorflow.org/api_docs/python/tf/device)
context manager.
:::

Now, import all necessary python libraries.

In [3]:
from glob import glob
import os
import pydicom as dicom
import tensorflow_mri as tfmri
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

### **Prepare the Data**
We will be using an open-source cardiac MRI  data set from Sunnybrook Cardiac Data (also known as the 2009 Cardiac MR Left Ventricle Segmentation Challenge data). It consists of 45 cine-MRI images from a mixed of patients and pathologies: healthy, hypertrophy, heart failure with infarction and heart failure without infarction. A subset of this data set was first used in the automated myocardium segmentation challenge from short-axis MRI, held by a MICCAI workshop in 2009, which is what we will be using in this tutorial:

*Radau P, Lu Y, Connelly K, Paul G, Dick AJ, Wright GA. “Evaluation Framework for Algorithms Segmenting Short Axis Cardiac MRI.” The MIDAS Journal – Cardiac MR Left Ventricle Segmentation Challenge, http://hdl.handle.net/10380/3070*



The  complete data set is  available in the CAP database with public domain license, and can be download from https://www.cardiacatlas.org/sunnybrook-cardiac-data/. Here the manual segmentations are stored as coordinates, however this same data set can be downloaded from Matlab, where the masks have already been created (see: https://uk.mathworks.com/help/deeplearning/ug/cardiac-left-ventricle-segmentation-from-cine-mri-images.html).

**Download Data Set**
Download the data set from the MathWorks® website and unzip the downloaded folder.

```
zipFile = matlab.internal.examples.downloadSupportFile("medical","CardiacMRI.zip");
filepath = fileparts(zipFile);
unzip(zipFile,filepath)
The imageDir folder contains the downloaded and unzipped data set.

imageDir = fullfile(filepath,"Cardiac MRI");
```

Copy the CardiacMRI folder to your workspace.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# specify your image path
image_path = '/content/drive/MyDrive/MLtutorials/CardiacMRI/'

This dataset consists of short-axis stacks from 45 patients, with manual segmetnations in the LV bloodpool at systole and diastole. The MRI images are in the DICOM file format (.dcm) and the label images are in the PNG file format.

First, we need to extract the data and store our image and mask pairs in lists, so that pairs have the same index. Our path names may differ
so ensure you the given paths with the path where your data is stored.

In [5]:
# This code works if the data was downloaded from MATLAB. If you took the original data, you will have to process to generate the masks from the coordiates

#Create empty lists to store the image and mask data
images = []
masks = []

#Obtain all image and mask data, may need to check the image and mask pathnames to see how they differ

# The images (.dcm) files are in the CardiacMRI folder in a folder named 'images' - here they are seperated by patient
for filename in glob(os.path.join(image_path,'images/**/*.dcm')):     # This will select all files ending with dcm - these should be your images

    #Now we have to find the matching mask data
    # These files are instead in a folder names 'labels'
    mask_filename = filename.replace('images','labels')                #Replacing the different words in pathnames
    # The filenames contain 'gtmask0' in place of '_rawdcm_'
    mask_filename_mask = mask_filename.replace('_rawdcm_','gtmask0')
    # and they are of type png not dcm
    mask_filename_png = mask_filename_mask.replace('dcm','png')        #replacing file type
    mask_exists = os.path.exists(mask_filename_png)                    #Checking that this path exists

    #If this path exists, we append the images and masks lists so that corresponding images and paths have the same index

    if mask_exists:
        image = dicom.dcmread(filename).pixel_array
        images.append(image)
        mask = mpimg.imread(mask_filename_png)
        masks.append(mask)


We now have 805 image-mask pairs.

Next, we will check if we have assigned the correct pairs of images and masks by plotting them on top of each other. Image cmap convention is 'gray' in MRI, and masks cmap is often 'viridis'. Alpha determines how in focus the mask is (range 0-1, 0.2 is a good choice)

In [ ]:
imageNo = 10

#testing blended images and masks
plt.imshow(images[imageNo],cmap='gray')
plt.imshow(masks[imageNo], cmap='viridis',alpha=0.2)
plt.title("Image With Mask")


In [ ]:
#converting to numpy arrays as they are easier to work with
masks = np.array(masks)
images = np.array(images)

# You should check the shape of the data sets.
print(f'The shape of the masks is: {masks.shape}')
print(f'The shape of the images is: {images.shape}')


From this we can see we have 805 images and masks, with image size 256x256, as we expect.

Next, we need to normalize the image data to so that each data point lies between 0-1.

In [8]:
def normalize_images(images):
    """Normalisation function
    Input:
           - images, array of image data
    Output:
           - normalised array of image data"""

    return (images - np.min(images)) / (np.max(images) - np.min(images))

#Creating new normalised array
images = normalize_images(images)

For a 2D Unet model, we need our mask data to be seperated in two classes, in this case background and bloodpool. All data points equal to 0 we can keep as zero, and all non zero points we will make equal to 1.

In [9]:
#simplifying masks
def simplify_mask(masks):
    """This function take an inputed array of masks and keeps leaves all zero data points as zero
    and converts all non-zero data points equal to one. Zero data point are equivalent to the background
    and those equal to one represent the bloodpool

    Input: masks, array containing mask data

    Output: simp_masks, array containing the simplified masks"""

    simp_masks = np.zeros_like(masks, dtype=np.float32)
    simp_masks[masks ==0] = 0        #background
    simp_masks[masks !=0] = 1        #bloodpool

    return simp_masks

We can now apply this function to our `masks` array, and do a quick check of the minimum and maximum values (which should be 0 and 1 respectively), and plot one of the simplified masks.

In [ ]:
#Simplifying all masks
masks = simplify_mask(masks)

#Test
print(f"(min, max): ({masks.min()}, {masks.max()})")

plt.imshow(masks[0], cmap="viridis")
plt.show()

Next, we need to seperate our data into training, testing and validation data. We will use the ratio of 80%, 10%, 10% respectively, although this is some what arbitrary. Ideally you should use as much of the data as possible for training, while keeping enough for testing and validation.

In [11]:
#seperating into training, testing and validation data

#Finding the number of images/masks in category
n_train = int(0.8*masks.shape[0])                         #80%
n_test =int(0.1*masks.shape[0])                           #10%
n_val = int(0.1*masks.shape[0] )                           #10%

#Note: rounded we will have 81 test data and 80 val (not exactly 10%)

#seperating the data into categories

train_images = images[:n_train]
train_masks = masks[:n_train]

test_images = images[n_train:n_train+n_test+1]
test_masks = masks[n_train:n_train+n_test+1]

val_images = images[n_train+n_test+1:]
val_masks = masks[n_train+n_test+1:]

We need to add a "channel dimension" to have our data in the correct format for tensorflow. We can do this by using the `np.expand_dims()` function. Set axis = 3

In [12]:
train_images = np.expand_dims(train_images,axis=3)
test_images = np.expand_dims(test_images,axis=3)
val_images = np.expand_dims(val_images,axis=3)

train_masks = np.expand_dims(train_masks,axis=3)
test_masks = np.expand_dims(test_masks,axis=3)
val_masks = np.expand_dims(val_masks,axis=3)

We now need to set our parameters for the model

In [13]:
IM_height = images.shape[1]     #Extracting the image height
IM_length = images.shape[2]     #Extracting the image length
channels = 1                    #Only need 1 input channel so set to 1
batch_size = 16                 #Arbitrary choice, you can try setting to different values to optimize the model
epochs = 100                     #ideally set to 100, depending on time and size of GPU. The larger the number of epochs the better, however increases execution time

We now need to convert our data into tensorflow (tf) datasets, and then seperate them into batches.

In [14]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
train_dataset = train_dataset.batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_masks))
val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images))
test_dataset = test_dataset.batch(batch_size)

The next cell is optional, however if you would like to save you model so that it can be used at a later date it is quite useful.
The code below will allow the model automatically save the best model. While the model is being build, it will only save itself if the validation loss has decreased compared to the model at any previous epoch.

In [15]:
#First create a pathname of where to save the model
checkpointpath = "./models/unet_segmentation/model.ckpt"
checkpoint_dir = os.path.dirname(checkpointpath)

#
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpointpath, save_best_only=True, verbose=1)

### Building Our Model
Now we are ready to make our model. TensorFlow MRI has a premade 2D Unet model function which can be called from
`tfmri.models.Unet2D()`. Details of this model and its unputs can be found on the TensorFlow MRI website.
The basic parameters we need to define are:

`filters=` - The number of filters for convolutional layers at each scale. The number of scales is inferred as len(filters). E.g. a list of 3 values/scales, where each value is double that of the last eg. `[32,64,128]`

`kernel_size=` - define the dimensions of the kernel as a list, if you only input 1 value then it will assume dimensions are the same size

`out_channels=` - should keep the same as the number of input channels, in this case =1

`out_activation=` - set this = `"sigmoid"` for segmentation task


We then need to compile our model. The basic parameters we need to define are:

`optimizer=` - normally set our optimizer = `"adam"`, more options available on [GitHub](https://mrphys.github.io/tensorflow-mri/)

`loss=` - many options which you can read about on [GitHub](https://mrphys.github.io/tensorflow-mri/), however we will use `"binary_crossentropy"`. You can test and
compare some of the other loss types.

`metrics=` - we will use `tfmri.metrics.Accuracy()`

In [16]:
model = tfmri.models.UNet2D(filters= [32,64,128], kernel_size = 3, out_channels=1, out_activation = "sigmoid")
model.compile(optimizer = 'adam', loss="binary_crossentropy", metrics=tfmri.metrics.Accuracy())

Now we will build our model to our data using the `fit()` function.
input:

`training_data=` - we use our `train_dataset`

`validation_data=` - we use our `val_dataset`

`batch_size=` - we use our `batch_size`

`verbose=` - we set = 1

`epochs=` - we use our `epochs`

`callbacks=` - this is what saves our model, we set to our variable `cp_callback`

Building our model will take some time  so you may want to test that it is working by first using a small number of epochs (even just 1)

In [ ]:
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    batch_size=batch_size,
                    verbose=1,
                    epochs=epochs,
                    callbacks=[cp_callback])

### Results
Our model should now be build, we can extract the loss and accuracy data and plot to see how they very accross epochs.

In [ ]:
#Exctracting data from our model
loss = history.history['loss']
accuracy = history.history['acc']
epochs = np.arange(1,len(loss)+1)

loss_range = len(loss)
val_loss = history.history['val_loss']

#Creating figures and plotting our losses and accuracy against epochs.
fig = plt.figure(figsize=(20,15))
ax1, ax2 = plt.subplot(2,2,1), plt.subplot(2,2,2)
ax1.plot(epochs,loss,'b',label= 'Loss')
ax1.plot(epochs,val_loss,'g', label = 'Validation Loss')
ax1.legend()
ax1.set_title('Cross Entropy Loss', fontsize='16')
ax1.set_xlabel('Epochs', fontsize = '14')
ax1.set_ylabel('Loss', fontsize = '14')

ax2.plot(epochs,accuracy,'r')
ax2.set_title('Accuracy', fontsize = '16')
ax2.set_xlabel('Epochs', fontsize = '14')
ax2.set_ylabel('Accuracy', fontsize = '14')

We can see that the losses decrease accross the epochs, and the accuracy increases.

Now we can apply our model to the `test_dataset` to inspect if the model is actual able to predict the masks of our test images.

In [ ]:
#Predict the masks of our test images
out_ims = model.predict(test_dataset)

We can now plot the ground truth test image and mask (the image with its premade mask) next to the image and generated mask to compare.

In [ ]:
i = 9
#pltting the above image index

figcompare = plt.figure(figsize=(16,12))
ax1, ax2, ax3, ax4 = plt.subplot(2,4,1), plt.subplot(2,4,2), plt.subplot(2,4,3), plt.subplot(2,4,4)

ax1.imshow(test_images[i],cmap='gray')
ax1.imshow(test_masks[i],alpha=0.2)
ax1.set_title("Test Image with Ground Truth Mask")

ax2.imshow(test_images[i], cmap='gray')
ax2.imshow(out_ims[i],alpha=0.2)
ax2.set_title("Test Image with Predicted Mask")

ax3.imshow(test_masks[i])
ax3.set_title('Ground Truth Mask')

ax4.imshow(out_ims[i])
ax4.set_title('Predicted Mask')


We can also plot the difference map between the ground truth mask and model generated mask

In [ ]:
#Difference in Mask
mask_diff = abs(out_ims[i] - test_masks[i])

plt.imshow(mask_diff)
plt.title("Difference Map")

# Conclusion
Congratulations! You've performed image segmentation using TensorFlow MRI. The code in this notebook will work with any 2D dataset, so feel free to use your own. You can also continue to test this model using different parameters, which can be found on [GitHub](https://github.com/mrphys/tensorflow-mri/issues/new).

### Let us know!
Please tell us what you think about this tutorial and about TensorFlow MRI.
We would like to hear what you liked and how we can improve. You will find us
on [GitHub](https://mrphys.github.io/tensorflow-mri/).

In [ ]:
# Copyright 2022 University College London. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.